In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings
from mp_functions import request_rates, request_ohlcv, compute_log_return
from mp_functions import request_cmc_historical
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

In [2]:
#load coin names - it removes those with some outliers in historical data
with open('/home/fbuonerba/codes/meta_data/new_coins.txt') as ff:
    coins=json.load(ff)
bad=['NPXS','MKR','VET','RHOC', 'ONT', 'ZIL', 'NANO', 'BAT','BCD','XTZ']
coins=np.array(coins)
where=[i for i in range(len(coins)) if coins[i] in bad]
coins=np.delete(coins, where)

prettybad=['BTC','BCN','DCR','BTG','BTM','XVG']
prettywhere=[i for i in range(len(coins)) if coins[i] in prettybad]
coins=np.delete(coins, prettywhere)
quotes=['BTC']
print(coins)

['ETH' 'XRP' 'BCH' 'EOS' 'XLM' 'LTC' 'ADA' 'XMR' 'IOTA' 'TRX' 'ETC' 'DASH'
 'NEO' 'XEM' 'BNB' 'ZEC' 'OMG' 'LSK' 'ZRX' 'QTUM' 'DOGE' 'BTS' 'DGB' 'ICX'
 'STEEM' 'AE' 'WAVES' 'SC' 'REP' 'PPT' 'GNT' 'STRAT']


In [4]:
#####MAINTAINANCE OF EXCHANGE RATES#######
pathname='/home/fbuonerba/exchange_rates_data/'
exch=os.listdir(pathname)
EX={}
DEX={}
for x in exch:
    X=x.split('.')
    X=X[0].split('_')
    X[-1]=int(X[-1])
    with open(pathname + x) as infile:
        try:
            ex=json.load(infile)
            if ex=={} or ex=='{}':
                print(x, ex)
                request_rates(X[-1], X[2], X[3])
        except Exception as e: 
            print(e, x)
            request_rates(X[-1], X[2], X[3])
    



#create a dictionary coin ---> exchange rates coin/BTC listed in directory
#it looks only for those dated after 1515000000 - January 3rd 2018.


# for coin in coins:
#     EX[coin]=[x for x in exch if (coin+'_' in x)]
#     DEX[coin]=[]
#     for x in EX[coin]:
#         try:
#             if float(x[-14:-4])>1515000000:
#                 DEX[coin].append(x)
#         except:
#             print('folder in wrong format - remove -----', x)
#             os.unlink(pathname+x)

exchange_rate_USDT_BTC_1430357000.txt {}
exchange_rate_BNB_USD_1543334400.txt {}
exchange_rate_NANO_USD_1540033200.txt {}
exchange_rate_VEN_USD_1497564000.txt {}
exchange_rate_REP_USD_1432769000.txt {}
exchange_rate_GNT_USD_1490857200.txt {}
exchange_rate_IOTA_USD_1440321800.txt {}
exchange_rate_ODE_BTC_1491512400.txt {}
exchange_rate_HB_BTC_1508313600.txt {}
exchange_rate_QTUM_USD_1493596800.txt {}
exchange_rate_VEN_USD_1515931200.txt {}
exchange_rate_LIZA_BTC_1497211200.txt {}
exchange_rate_AIO_BTC_1512367200.txt {}
exchange_rate_HB_BTC_1483574400.txt {}
exchange_rate_LRC_BTC_1483538400.txt {}
exchange_rate_BNB_USD_1525824000.txt {}
exchange_rate_LRC_USD_1515650400.txt {}
exchange_rate_STEEM_USD_1541552400.txt {}
exchange_rate_DASH_USD_1440444200.txt {}
exchange_rate_TRX_BTC_1485712800.txt {}
exchange_rate_BAT_USD_1438035800.txt {}
exchange_rate_IOTA_USD_1488844800.txt {}
exchange_rate_XEM_USD_1491022800.txt {}
exchange_rate_LRC_BTC_1503820800.txt {}
exchange_rate_BAT_USD_1441275800.

exchange_rate_BNB_USD_1526358200.txt {}
exchange_rate_EMC_BTC_1487541600.txt {}
exchange_rate_DAT_BTC_1500228000.txt {}
exchange_rate_EOS_BTC_1485864000.txt {}
exchange_rate_SNT_USD_1509040800.txt {}
exchange_rate_LSK_USD_1504616400.txt {}
exchange_rate_ODE_BTC_1520568000.txt {}
exchange_rate_XMR_USD_1436379800.txt {}
exchange_rate_XLM_BTC_1430454200.txt {}
exchange_rate_DAT_USD_1439339000.txt {}
exchange_rate_BCN_USD_1543230000.txt {}
exchange_rate_RLC_BTC_1486778400.txt {}
exchange_rate_ODE_BTC_1504976400.txt {}
exchange_rate_VEN_USD_1492628400.txt {}
exchange_rate_DIG_USD_1513137600.txt {}
exchange_rate_ZEC_USD_1438161800.txt {}
exchange_rate_LRC_BTC_1500346800.txt {}
exchange_rate_BTC_BTC_1529852400.txt {}
exchange_rate_RLC_USD_1433730200.txt {}
exchange_rate_BCH_BTC_1501570800.txt {}
exchange_rate_BNB_USD_1461108200.txt {}
exchange_rate_IOTA_BTC_1494421200.txt {}
exchange_rate_LIZA_BTC_1503518400.txt {}
exchange_rate_LIZA_BTC_1430594600.txt {}
exchange_rate_ADA_USD_1518053000.txt 

exchange_rate_EOS_USD_1491483600.txt {}
exchange_rate_SNT_USD_1490144400.txt {}
exchange_rate_NEO_BTC_1490979600.txt {}
exchange_rate_LIZA_USD_1430540600.txt {}
exchange_rate_ZRX_BTC_1484686800.txt {}
exchange_rate_PPT_USD_1540357200.txt {}
exchange_rate_LIZA_USD_1500537600.txt {}
exchange_rate_BTG_USD_1436671400.txt {}
exchange_rate_ELF_BTC_1500742800.txt {}
exchange_rate_HB_BTC_1486242000.txt {}
exchange_rate_HB_USD_1435127000.txt {}
exchange_rate_QTUM_BTC_1485162000.txt {}
exchange_rate_BNB_USD_1456039400.txt {}
exchange_rate_LSK_USD_1509829200.txt {}
exchange_rate_VEN_USD_1440055400.txt {}
exchange_rate_HBZ_USD_1432556600.txt {}
exchange_rate_ODE_USD_1493456400.txt {}
exchange_rate_BNB_USD_1540738800.txt {}
exchange_rate_LIZA_BTC_1516237200.txt {}
exchange_rate_GNT_USD_1498284000.txt {}
exchange_rate_RLC_USD_1483293600.txt {}
exchange_rate_MKR_BTC_1504537200.txt {}
exchange_rate_XVG_USD_1505242800.txt {}
exchange_rate_DIG_USD_1431271400.txt {}
exchange_rate_NEO_BTC_1487746800.txt {

KeyboardInterrupt: 

In [24]:

#This window scans through the directory contiaing exchange rates and looks for
#wrong data. It then re-downloads data from coinAPI.
#REMARK: it seems BNB has real missing data.

#loop over dictionary values, to find wrong data (empty folder). Then download it.
for coin in coins:
    for exc in DEX[coin]:
        X=exc.split('.')
        X=X[0].split('_')
        X[-1]=int(X[-1])
        with open(pathname + exc) as infile:
            try:
                ex=json.load(infile)
                if ex=={} or ex=='{}':
                    print(exc, ex)
                    request_rates(X[-1], X[2], X[3])
            except Exception as e: 
                print(e, exc)
                request_rates(X[-1], X[2], X[3])
            

exchange_rate_BNB_BTC_1518058800.txt {}
exchange_rate_BNB_BTC_1529985600.txt {}
exchange_rate_BNB_BTC_1530108000.txt {}
exchange_rate_BNB_BTC_1518062400.txt {}
exchange_rate_BNB_BTC_1518098400.txt {}
exchange_rate_BNB_BTC_1518051600.txt {}
exchange_rate_BNB_BTC_1518066000.txt {}
exchange_rate_BNB_BTC_1529992800.txt {}
exchange_rate_BNB_BTC_1518084000.txt {}
exchange_rate_BNB_BTC_1530007200.txt {}
exchange_rate_BNB_BTC_1518094800.txt {}
exchange_rate_BNB_BTC_1529989200.txt {}
exchange_rate_BNB_BTC_1518116400.txt {}
exchange_rate_BNB_BTC_1529996400.txt {}
exchange_rate_BNB_BTC_1530014400.txt {}
exchange_rate_BNB_BTC_1530003600.txt {}
exchange_rate_BNB_BTC_1518141600.txt {}
exchange_rate_BNB_BTC_1529982000.txt {}
exchange_rate_BNB_BTC_1517572800.txt {}
exchange_rate_BNB_BTC_1518069600.txt {}
exchange_rate_BNB_BTC_1518127200.txt {}
exchange_rate_BNB_BTC_1518055200.txt {}
exchange_rate_BNB_BTC_1518087600.txt {}
exchange_rate_BNB_BTC_1518152400.txt {}
exchange_rate_BNB_BTC_1518145200.txt {}


In [25]:
########MAINTAINANCE OF LOG-RETURNS##########
pathname1='/home/fbuonerba/log_returns_data/'
log=os.listdir(pathname1)
LOG={}
DLOG={}
#create a dictionary coin --> list of log-returns folders for pair coin/BTC.
#it looks only for those dated after 1515000000 - January 3rd 2018.
for coin in coins:
    LOG[coin]=[x for x in log if (coin+'_BTC' in x)]
    DLOG[coin]=[]
    for x in LOG[coin]:
        try:
            if float(x[-14:-4])>1515000000:
                DLOG[coin].append(x)
        except:
            print('folder in wrong format - remove -----', x)
            os.unlink(pathname1+x)

In [26]:
#maintainance of daily log-returns. 
#This window scans through the directory contiaing log-returns and looks for
#corrupted data. It then re-computes daily log-returns.
#REMARK: there are also many hourly log-returns. This window does not deal with them.

#loop over dictionary values, to find wrong data,by default NaN values. Then re-compute.
for coin in coins:
    for log in DLOG[coin]:
        X=log.split('.')
        X=X[0].split('_')
        X[-1]=int(X[-1])
        X[-2]=int(X[-2])
        with open(pathname1 + log) as infile:
            lg=json.load(infile)
            if np.isnan(lg)==True and X[-1]-X[-2]==86400:#focus on daily returns.
                print(log, lg)
                compute_log_return(X[-1], coin, 'BTC', X[-1]-X[-2])
                
                

log_return_BNB_BTC_1529989200_1530075600.txt nan
log_return_BNB_BTC_1529985600_1530072000.txt nan
log_return_BNB_BTC_1518116400_1518202800.txt nan
log_return_BNB_BTC_1530003600_1530090000.txt nan
log_return_BNB_BTC_1530108000_1530194400.txt nan
log_return_BNB_BTC_1529982000_1530068400.txt nan
log_return_BNB_BTC_1518098400_1518184800.txt nan
log_return_BNB_BTC_1518094800_1518181200.txt nan
log_return_BNB_BTC_1518091200_1518177600.txt nan
log_return_BNB_BTC_1530000000_1530086400.txt nan
log_return_BNB_BTC_1518073200_1518159600.txt nan
log_return_BNB_BTC_1518055200_1518141600.txt nan
log_return_BNB_BTC_1530007200_1530093600.txt nan
log_return_BNB_BTC_1518145200_1518231600.txt nan
log_return_BNB_BTC_1530014400_1530100800.txt nan
log_return_BNB_BTC_1518152400_1518238800.txt nan
log_return_BNB_BTC_1518051600_1518138000.txt nan
log_return_BNB_BTC_1518102000_1518188400.txt nan
log_return_BNB_BTC_1518062400_1518148800.txt nan
log_return_BNB_BTC_1518127200_1518213600.txt nan
log_return_BNB_BTC_1

In [27]:
#This window check that every daily return is actually present in directory.
t0=1515024000 #jan 4th.
TIME=int((time.time()-t0)/86400 - 1)
for coin in coins:
    for t in range(TIME):
        init=t0+t*86400
        end=t0+(t+1)*86400
        symbol='log_return_'+coin+'_BTC_'+str(init)+'_'+str(end)+'.txt'
        if symbol not in DLOG[coin]:
            print(symbol)
            compute_log_return(init, coin, 'BTC', 86400)
    

In [ ]:
########MAINTAINANCE OF OHLCV##########
pathname3='/home/fbuonerba/ohlcv_data/'
ohlcv=os.listdir(pathname3)

OH={}
DOH={}
#create a dictionary coin ---> all ohlcv coin/BTC (for every exchange) listed in directory
#it looks only for those dated after 1515000000 - January 3rd 2018.
for coin in coins:
    OH[coin]=[x for x in ohlcv if (coin+'_BTC' in x)]
    DOH[coin]=[]
    for x in OH[coin]:
        if float(x[-14:-4])>1515000000:
            DOH[coin].append(x)

In [ ]:
#This window scans through directory to find missing ohlcv data, and in case downloads it. 
#Almost always, exchange QRYPTOS has missing data.
#DASH,NEO,XEM have a quite a few missing data points.
for coin in coins:
    for x in DOH[coin]:
        with open(pathname3 + x) as infile:
            oh=json.load(infile)
        if oh==[] or oh=='[]':
            x=x.split('.')
            x=x[0].split('_')
            x[-1]=int(x[-1])
            x[-2]=int(x[-2])
            if x[1]!='QRYPTOS':
                print(x)
                #request_ohlcv(x[-2], x[1]+'_'+x[2]+'_'+x[3]+'_BTC',1)

In [22]:
#########MAINTAINANCE OF COIN SUPPLY###########
#historical daily cmc data is impossible to correct because it is taken live.
#We can nevertheless check and eventually manually adjust.
pathname4='/home/fbuonerba/cmc_data/'
cmc=os.listdir(pathname4)
DCMC={}
for coin in coins:
    DCMC[coin]=[x for x in cmc if (coin in x)]
    for x in DCMC[coin]:
        try:
            with open(pathname4 + x) as infile:
                cm=json.load(infile)
            if cm=='{}' or cm=={}:
                print(x, cm)
            elif np.isnan(cm['circulating_supply'])==True:
                print('circ', x, cm)
        except:
            print(x)
            #only supply matters - can be manually filled.


In [6]:
#weekly cmc can be re-downloaded. If there are empty folders, or some folders
#contains NaN values, this window re-downloads the data for that whole day.
WEEK=[x for x in cmc if 'historical' in x]
WEEK.sort()
for x in WEEK:
    test=0
    X=x.split('.')
    ttime=float(X[0].split('_')[-1])
    utctime = datetime.utcfromtimestamp(ttime).strftime('%Y%m%d')
    with open(pathname4 + x) as infile:
        try:
            hist=json.load(infile)
            for coin in coins:
                if coin=='IOTA':
                    coin='MIOTA'
                if np.isnan(hist[coin]['supply'])==True:
                    print(coin, x)
                    test=1
            if test==1:
                request_cmc_historical(int(utctime))
        except:
            print('missing day', x)
            request_cmc_historical(int(utctime))